Stratégie “méta” : partir d’une allocation low-vol et essayer de détecter des jours “spéciaux” (risk-on / risk-off) à partir de features de marché. La différence ici, c’est qu’au lieu d’une régression logistique, on utilise un modèle XGBoost (plus flexible) pour estimer deux probabilités : risque de forte baisse (queue basse) et chance de forte hausse (queue haute). Ensuite, sur la fenêtre de test (365 derniers jours), on transforme ces probabilités en régimes d’allocation (baseline low-vol, equal-weight ou cash) et on backteste la perf nette en incluant des coûts via le turnover, puis on compare au baseline.

In [ ]:
import numpy as np
import pandas as pd
from sklearn.metrics import accuracy_score
import matplotlib.pyplot as plt

from xgboost import XGBClassifier

# params
Q_MOVE = 0.30
TEST_DAYS = 365
COST_BPS = 10

# features stack (reprend rets/close/vol déjà calculés plus haut)
# si tu lances cellule B seule: rebuild rets/close/vol comme avant

WIN_MOM1, WIN_MOM2, WIN_MOM3 = 5, 20, 60
LAM = 0.94
WIN_VOL = 20

vol_ewma = np.sqrt((rets**2).ewm(alpha=1-LAM).mean())
vol_roll = rets.rolling(WIN_VOL).std()
mom5  = close.pct_change(WIN_MOM1)
mom20 = close.pct_change(WIN_MOM2)
mom60 = close.pct_change(WIN_MOM3)

# dataset long format
X = pd.concat([
    rets.stack().rename("ret_1"),
    mom5.stack().rename("ret_5"),
    mom20.stack().rename("ret_20"),
    mom60.stack().rename("ret_60"),
    vol_ewma.stack().rename("vol_ewma"),
    vol_roll.stack().rename("vol_roll20"),
    (vol_ewma / vol_ewma.rolling(252).mean()).stack().rename("vol_ratio"),
], axis=1).dropna()

fut = rets.shift(-1).stack().rename("ret_fut")
df_all = pd.concat([X, fut], axis=1).dropna()

down_thr = df_all["ret_fut"].quantile(Q_MOVE)
up_thr   = df_all["ret_fut"].quantile(1 - Q_MOVE)

df_all["y_down"] = (df_all["ret_fut"] <= down_thr).astype(int)
df_all["y_up"]   = (df_all["ret_fut"] >= up_thr).astype(int)

cutoff = df_all.index.get_level_values(0).max() - pd.Timedelta(days=TEST_DAYS)
train = df_all[df_all.index.get_level_values(0) < cutoff].copy()
test  = df_all[df_all.index.get_level_values(0) >= cutoff].copy()

X_train = train.drop(columns=["ret_fut","y_down","y_up"])
X_test  = test.drop(columns=["ret_fut","y_down","y_up"])
y_down_train = train["y_down"]
y_up_train   = train["y_up"]
y_down_test  = test["y_down"]
y_up_test    = test["y_up"]

# model params (safe)
params = dict(
    n_estimators=400,
    max_depth=3,
    learning_rate=0.05,
    subsample=0.8,
    colsample_bytree=0.8,
    reg_lambda=1.0,
    min_child_weight=5,
    objective="binary:logistic",
    eval_metric="logloss",
    n_jobs=-1,
    random_state=42,
)

m_down = XGBClassifier(**params)
m_up   = XGBClassifier(**params)

m_down.fit(X_train, y_down_train)
m_up.fit(X_train, y_up_train)

# proba
train_pdown = m_down.predict_proba(X_train)[:, 1]
train_pup   = m_up.predict_proba(X_train)[:, 1]
test["p_down"] = m_down.predict_proba(X_test)[:, 1]
test["p_up"]   = m_up.predict_proba(X_test)[:, 1]

# threshold calibration (train)
# top tail -> act only on high confidence
THR_DOWN = float(np.quantile(train_pdown, 0.95))
THR_UP   = float(np.quantile(train_pup,   0.95))
print("thr_down", THR_DOWN, "thr_up", THR_UP)

# meta mode per row
test["mode"] = "baseline"
test.loc[test["p_down"] >= THR_DOWN, "mode"] = "risk_off"
test.loc[(test["mode"] == "baseline") & (test["p_up"] >= THR_UP), "mode"] = "risk_on"

# mode by date (mean)
mode_by_date = (
    test.reset_index()
        .groupby("timestamp")[["p_down","p_up","mode"]]
        .agg({"p_down":"mean","p_up":"mean","mode":lambda x: x.iloc[0]})
)

dates_test = mode_by_date.index

# weights templates
# baseline = low_vol
w_low = (vol_roll.le(vol_roll.quantile(0.5, axis=1), axis=0)).astype(float)
w_low = w_low.reindex(dates_test).fillna(0.0)

# risk_on = equal weight
w_on = pd.DataFrame(1.0, index=dates_test, columns=SYMBOLS)

# risk_off = cash
w_off = pd.DataFrame(0.0, index=dates_test, columns=SYMBOLS)

# build W
W = pd.DataFrame(index=dates_test, columns=SYMBOLS, dtype=float)
for d in dates_test:
    m = mode_by_date.loc[d, "mode"]
    if m == "risk_off":
        W.loc[d] = w_off.loc[d]
    elif m == "risk_on":
        W.loc[d] = w_on.loc[d]
    else:
        W.loc[d] = w_low.loc[d]

W = W.clip(lower=0.0)
W = W.div(W.sum(axis=1).replace(0, np.nan), axis=0).fillna(0.0)

# backtest test window
rets_test = rets.reindex(dates_test)

def run_bt(weights, rets, cost_bps=10):
    w = weights.shift(1).reindex(rets.index).fillna(0.0)
    w = w.div(w.sum(axis=1).replace(0, np.nan), axis=0).fillna(0.0)
    gross = (w * rets).sum(axis=1)
    to = w.diff().abs().sum(axis=1).fillna(0.0)
    cost = to * (cost_bps / 10000.0)
    net = gross - cost
    return net, to

def stats(r, freq=365):
    r = r.dropna()
    eq = (1 + r).cumprod()
    mu = r.mean() * freq
    sig = r.std() * np.sqrt(freq)
    sharpe = mu / sig if sig > 0 else np.nan
    dd = (eq / eq.cummax() - 1.0).min()
    ann = eq.iloc[-1] ** (freq/len(r)) - 1 if len(r) > 0 else np.nan
    return float(ann), float(sig), float(sharpe), float(dd)

r_meta, to_meta = run_bt(W, rets_test, cost_bps=COST_BPS)
r_base, to_base = run_bt(w_low, rets_test, cost_bps=COST_BPS)

print("baseline test:", stats(r_base), "turn", float(to_base.mean()))
print("xgb meta test:", stats(r_meta), "turn", float(to_meta.mean()))
print("modes:", mode_by_date["mode"].value_counts().to_dict())

plt.figure(figsize=(12,5))
plt.plot((1+r_base.fillna(0)).cumprod(), label="baseline low_vol")
plt.plot((1+r_meta.fillna(0)).cumprod(), label="xgb meta")
plt.legend()
plt.title("Test equity")
plt.tight_layout()
plt.show()

thr_down 0.4721072018146515 thr_up 0.4820227026939392

baseline test: (-0.27284050251585945, 0.5103395328575311, -0.36612463641141907, -0.4492806842792576) turn 0.07741347905282331

xgb meta test: (-0.2934218385645678, 0.5112396706414283, -0.4206347012988575, -0.4492806842792576) turn 0.09198542805100182

modes: {'baseline': 363, 'risk_on': 3}

![Image test 10](image/image_test_10.png)


Sur la période test, le baseline low-vol est négatif (Sharpe < 0), et la version XGBoost méta fait légèrement pire, avec un turnover un peu plus élevé. Le compteur des modes montre que le modèle reste quasiment tout le temps en baseline (363 jours) et déclenche très rarement un signal risk_on (3 jours), et jamais de risk_off. 

Cela veut dire que même avec un modèle plus puissant, le signal est soit trop “timide” (seuils élevés via quantile 95%), soit que les features actuelles ne donnent pas une séparation claire des gros mouvements exploitables. 

Résultat : on ajoute surtout de la complexité/turnover sans améliorer la perf sur cette fenêtre. Une suite logique serait de tester d’autres seuils (quantile 90% par ex) ou une autre définition des “queues” (ex : Q_MOVE = 0.10).